In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib.colors import ListedColormap

from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from itertools import combinations

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer, AerSimulator
from qiskit.quantum_info import Statevector

# Layerwise Quantum DRL
1. Embed UAV memory experience replay parameter data into qubits 1-N.
2. M Layers for updating the parameter theta for gradient descent algorithm using ansatz for LQ-DRL.
3. Entire system works like PPO with a $\theta$ parameter that's updated using gradient descent (computed classically).
3. Gradient descent algorithm provides the updated parameter $\theta$ for the LQ-DRL ansatz.
4. Update $\theta$ in the angle rotation gates for more accurate LQ-DRL computation. 
5. Repeat until convergence of optimal energy efficiency, secrecy rate, etc. 

# Data to be Embedded
Must embed data in $n$ qubits $0, 1 \dots, N-2, N-1$. Table of data to be embedded into the ansatz (quantum circuit) are the following.

\begin{array} {|r|r|}\hline 0_0 & 0_1 & 0_2 \\ \hline 1_0 & 1_1 & 1_2 \\ \hline 2_0 & 2_1 & 2_2 \\ \hline 3_0 & 3_1 & 3_2 \\ \hline 4_0 & 4_1 & 4_2 \\ \hline 5_0 & 5_1 & 5_2 \\ \hline 6_0 & 6_1 & 6_2 \\ \hline 7_0 & 7_1 & 7_2 \\ \hline 8_0 & 8_1 & 8_2 \\ \hline 9_0 & 9_1 & 9_2 \\ \hline 10_0 & 10_1 & 10_2 \\ \hline 11_0 & 11_1 & 11_2 \\ \hline 12_0 & 12_1 & 12_2 \\ \hline 13_0 & 13_1 & 13_2 \\ \hline 14_0 & 14_1 & 14_2 \\ \hline 15_0 & 15_1 & 15_2 \\ \hline 16_0 & 16_1 & 16_2 \\ \hline 17_0 & 17_1 & 17_2 \\ \hline 18_0 & 18_1 & 18_2 \\ \hline 19_0 & 19_1 & 19_2 \\ \hline 20_0 & 20_1 & 20_2 \\ \hline 21_0 & 21_1 & 21_2 \\ \hline 22_0 & 22_1 & 22_2 \\ \hline 23_0 & 23_1 & 23_2 \\ \hline 24_0 & 24_1 & 24_2 \\ \hline 25_0 & 25_1 & 25_2 \\ \hline  \end{array}

# Reward Shaping Function
Reward should be allocated based on the energy efficiency $\eta(t)$ if $R_{k, n}^{sec} \ge R_{min}^{sec}$, $\forall k$. 
\
$
\begin{align}
    R(t) = 
    \begin{cases}
        \eta(t), if R_{k, n}^{sec} \ge R_{min}^{sec}, \forall k \\
        0, otherwise
    \end{cases}
\end{align}
$

# Layerwise-Quantum Deep Reinforcement Learning
From Silviranti et. al (2025), the actor network takes 12 encoded inputs with a phase-based encoding scheme for the data. 
For ever $N$ element to be encoded, $N$ qubits will be required. 
For angle encoding, normalise using MinMaxScaler() so that all parameters in the input vector are normalised in the bound of [-1, 1]. 


# Encoding Operation
$
S_{\textbf{x}}: \begin{Bmatrix} 
x_{n}
\end{Bmatrix}_{n=1}^{N}
\xrightarrow[]{} 
\begin{Bmatrix}
\phi_{n}
\end{Bmatrix}_{n=1}^{N}
$

Encoding involves the use of $RX$ gates with the input data scaled by $tanh$, i.e., $\theta_{n} = tanh(x_{n}), \forall n$ and the encoding operation can be expressed as $S_{x} = \bigotimes^{N}_{n=1} RX(tanh(\theta_{n}))$. 

Parameter vector update for $\theta_{g}$, where $G$ is the number of weighted parameters is $S_{\theta_g} = RY(tanh(\theta_{g}))$. This value is fed back into the Layerwise Quantum Embedding portion of the quantum circuitry after the encoding operation $S_{\textbf{x}}$ has occurred. 

# Layerwise Quantum Embedding 
$N = $ Number of inputs 

$M = $ Number of Layers

$G = $ Number of weighted parameters

$
U_{LQ}^{(1)} (\theta^{(0)}) = \bigotimes_{g=1}^{G} \bigotimes_{n=1}^{N} (S_{\theta_{g}^{(0)}}^{(1)}) \begin{pmatrix}
\Pi_{n=1}^{N} CZ(\phi_{2}^{(0)}|\phi_{1}^{(0)}) \otimes \dots \otimes CZ(\phi_{N}^{(0)}|\phi_{N-1}^{(0)})
\end{pmatrix} (S_{x_{n}}^{(1)})H
$

First layer contains Hadamard gate. Layers $2 \le m \le M$ are made up of the same operations, however, they do not contain the Hadamard gate operation.
Embedding of layers m = 2 to M can be expressed in the following manner:

$
U_{LQ}^{(2 \le m \le M)} (\theta^{(m-1)}) = \bigotimes_{m=2}^{M} \bigotimes_{g=1}^{G} \bigotimes_{n=1}^{N} (S_{\theta_{g}^{(0)}}^{(m)}) \begin{pmatrix}
\Pi_{m=1}^{M} \Pi_{n=1}^{N} CZ(\phi_{2}^{(m)}|\phi_{1}^{(m)}) \otimes \dots \otimes CZ(\phi_{N}^{(m)}|\phi_{N-1}^{(m)})
\end{pmatrix} (S_{x_{n}}^{(m)})
$

This expression denotes the embedding and parameter updating from the classical gradient descent computation, which is computed using the parameter-shift rule. 

# Decoding Operation 
The decoding operation involves the basis transformation of a quantum state $\phi_n$ to the Z-basis for measurement, i.e., the computational state basis consisting of $\ket{0}$ and $\ket{1}$ from measurement, which is followed by the decoding operation. 
The quantum measurement operation can be expressed as 

$
J^{(1)} (\theta_{g}^{(0)}) = Z(\phi_{n}^{(1)})
$

The measurement must occur $K_{shot}$ number of times. 

The decoding operation can be expressed as 

$
y \xleftarrow[]{} \frac{1}{K_{shot}} \sum_{k=1}^{K_{shot}} Z(\ket{\phi})
$

# Local Loss Training


# Parameter Shift Rule for Gradient Descent Computation


# Actor Network
The UAVs are multi-agents/actors in the DRL algorithm. 
They determine the optimal policy based on the observed state $s$ at timestep $t$. 
The optimal policy $a_{\pi}$ must be determined by the actor network $\pi(s|\theta^{\pi})$. 
A reward $r$ is calculated based on $a_{\pi}$ at the end of every episode and the next state $s'$ is used to update $s$. 

# Critic Network


# TODO List - Silvirianti et. al (2025) Implementation
1. Design & integrate the different layers in the quantum computation for the layerwise embedding process.
2. Design the decoding operation for measurement after each layer (measurement in Z-basis, this might require some basis transformations to work).
3. Implement the gradient descent algorithm for generating $\theta_{g}$ after each layer with the use of the parameter shift rule.
4. Determine what parameters will be required for the actor network
5. Determine what parameters will be required for the critic network

# TODO List - Thesis Implementation
1. Perform amplitude encoding on the input vector of data such that the number of qubits can be smaller than the number of data points from the input vector to be embedded. 
   - $N$ qubits can be used to encode $2^N$ data points
   - As complexity of $U_{LQ}^{(0)} (\theta^{(0)})$ as proposed by Silvirianti et. al is $O(N)$, this will decrease the computational complexity of the embedding operation such that it will be $O(log_2(N))$
2. Investigate if the number of $M$ layers can be optimised and see if for above a certain value of $M$ if the performance begins to degrade. 

In [ ]:
# TODO: Determine what the input data vector X should contain exactly
x = []
theta_arr = []
n_act_in = 12

qc = QuantumCircuit(n_act_in, n_act_in)
for i in range(n_act_in):
    theta = np.tanh(x[i])
    theta_arr.append(theta)
    qc.rx(theta, i)

In [2]:
# Function to generate the first layer including the quantum embedding of the G weighted parameters
# In this case, G = N, so including M & G as parameters to the function is somewhat redundant 
# Function should only be called for the 1st layer
# Theta parameters for the RX & RY gates are stored in a vector of N & G elements, respectively 
def lq_embed_1(qc, N, theta, theta_par):
    for h in range(N):
        qc.h(h)
    for n in range(N):
        qc.rx(theta[n], n)
    for o in range(N-1):
        qc.cz(o, o+1)
    for g in range(N):
        qc.ry(theta_par[g], g)
    return qc

In [16]:
# Function to encode the data and include the parameter updates from the gradient descent algorithm
# M layers with N qubits, i.e., the circuit has an MxN quantum volume 
# Function should only be called once during each layer 
def lq_embed_m(qc, N, theta, theta_par):
    for n in range(N):
        qc.rx(theta[n], n)
    for o in range(N-1):
        qc.cz(o, o+1)
    for g in range(N):
        qc.ry(theta_par[g], g)
    return qc

In [23]:
n_qubits = 13
m_layers = 3
qc = QuantumCircuit(n_qubits, n_qubits)
theta_arr = []
theta_par_arr = []

for i in range(0, n_qubits):
    theta_arr.append((i / n_qubits) * np.pi)
    theta_par_arr.append((i / n_qubits) * np.pi)

lqdrl_1 = lq_embed_1(qc, n_qubits, theta_arr, theta_par_arr)
print(lqdrl_1)

      ┌───┐  ┌───────┐      ┌───────┐                                      »
 q_0: ┤ H ├──┤ Rx(0) ├────■─┤ Ry(0) ├──────────────────────────────────────»
      ├───┤ ┌┴───────┴─┐  │ └───────┘┌──────────┐                          »
 q_1: ┤ H ├─┤ Rx(π/13) ├──■─────■────┤ Ry(π/13) ├──────────────────────────»
      ├───┤┌┴──────────┤        │    └──────────┘┌───────────┐             »
 q_2: ┤ H ├┤ Rx(2π/13) ├────────■─────────■──────┤ Ry(2π/13) ├─────────────»
      ├───┤├───────────┤                  │      └───────────┘┌───────────┐»
 q_3: ┤ H ├┤ Rx(3π/13) ├──────────────────■────────────■──────┤ Ry(3π/13) ├»
      ├───┤├───────────┤                               │      └───────────┘»
 q_4: ┤ H ├┤ Rx(4π/13) ├───────────────────────────────■────────────■──────»
      ├───┤├───────────┤                                            │      »
 q_5: ┤ H ├┤ Rx(5π/13) ├────────────────────────────────────────────■──────»
      ├───┤├───────────┤                                                   »

In [17]:
qc_m = QuantumCircuit(n_qubits, n_qubits)

lqdrl_m = lq_embed_m(qc_m, n_qubits, theta_arr, theta_par_arr)
print(lqdrl_m)

        ┌───────┐      ┌───────┐                                      »
 q_0: ──┤ Rx(0) ├────■─┤ Ry(0) ├──────────────────────────────────────»
       ┌┴───────┴─┐  │ └───────┘┌──────────┐                          »
 q_1: ─┤ Rx(π/13) ├──■─────■────┤ Ry(π/13) ├──────────────────────────»
      ┌┴──────────┤        │    └──────────┘┌───────────┐             »
 q_2: ┤ Rx(2π/13) ├────────■─────────■──────┤ Ry(2π/13) ├─────────────»
      ├───────────┤                  │      └───────────┘┌───────────┐»
 q_3: ┤ Rx(3π/13) ├──────────────────■────────────■──────┤ Ry(3π/13) ├»
      ├───────────┤                               │      └───────────┘»
 q_4: ┤ Rx(4π/13) ├───────────────────────────────■────────────■──────»
      ├───────────┤                                            │      »
 q_5: ┤ Rx(5π/13) ├────────────────────────────────────────────■──────»
      ├───────────┤                                                   »
 q_6: ┤ Rx(6π/13) ├─────────────────────────────────────────────

In [ ]:
# Function to scale the vector of input data (x) following the distribution of the hyperbolic tangent
def x_angle_transformation(N, inp_vec):
    x_arr = []
    for i in range(N):
        x_arr.append(np.tanh(inp_vec[N]))
    return x_arr

In [ ]:
# Function to generate the first layer including the quantum embedding of the G weighted parameters
# In this case, G = N, so including M & G as parameters to the function is somewhat redundant 
# Function should only be called for the 1st layer
# Theta parameters for the RX & RY gates are stored in a vector of N & G elements, respectively 
def lq_embed_1(qc, N, theta, theta_par):
    for h in range(N):
        qc.h(h)
    for n in range(N):
        qc.rx(theta[n], n)
    for o in range(N-1):
        qc.cz(o, o+1)
    for g in range(N):
        qc.ry(theta_par[g], g)
    return qc

In [ ]:
# Function to encode the data and include the parameter updates from the gradient descent algorithm
# M layers with N qubits, i.e., the circuit has an MxN quantum volume 
# Function should only be called once during each layer 
def lq_embed_m(qc, N, theta, theta_par):
    for n in range(N):
        qc.rx(theta[n], n)
    for o in range(N-1):
        qc.cz(o, o+1)
    for g in range(N):
        qc.ry(theta_par[g], g)
    return qc

In [28]:
def build_ansatz(N, M, theta_arr, theta_par_arr):
    qc = QuantumCircuit(N)
    # First layer
    qc = lq_embed_1(qc, N, theta_arr, theta_par_arr)
    # Remaining M-1 layers
    for m in range(1, M):
        qc = lq_embed_m(qc, N, theta_arr, theta_par_arr)
    return qc

In [ ]:
# TODO: INPUT DATA VECTOR SHOULD BE MADE HERE
# Pass this vector to the x_angle_transformation() function and then pass that function to the build_ansatz() function

In [29]:
lqdrl = build_ansatz(n_qubits, m_layers, theta_arr, theta_par_arr)
print(lqdrl)

      ┌───┐  ┌───────┐      ┌───────┐ ┌───────┐                            »
 q_0: ┤ H ├──┤ Rx(0) ├────■─┤ Ry(0) ├─┤ Rx(0) ├─────────────────────■──────»
      ├───┤ ┌┴───────┴─┐  │ └───────┘┌┴───────┴─┐ ┌──────────┐      │      »
 q_1: ┤ H ├─┤ Rx(π/13) ├──■─────■────┤ Ry(π/13) ├─┤ Rx(π/13) ├──────■──────»
      ├───┤┌┴──────────┤        │    └──────────┘┌┴──────────┤┌───────────┐»
 q_2: ┤ H ├┤ Rx(2π/13) ├────────■─────────■──────┤ Ry(2π/13) ├┤ Rx(2π/13) ├»
      ├───┤├───────────┤                  │      └───────────┘├───────────┤»
 q_3: ┤ H ├┤ Rx(3π/13) ├──────────────────■────────────■──────┤ Ry(3π/13) ├»
      ├───┤├───────────┤                               │      └───────────┘»
 q_4: ┤ H ├┤ Rx(4π/13) ├───────────────────────────────■────────────■──────»
      ├───┤├───────────┤                                            │      »
 q_5: ┤ H ├┤ Rx(5π/13) ├────────────────────────────────────────────■──────»
      ├───┤├───────────┤                                                   »

In [ ]:
# Function to perform the measurement operation (Z-basis transformation) on the quantum state output of the layerwise quantum embedding 
def meas_op():
    J = 0
    return J

In [ ]:
# Function to perform the decoding operation on the output of the measurement operator 
def decode_op():
    y = 0
    return y

In [ ]:
# Function to perform the computation of the parameter shift rule for the gradient descent computation after the layerwise quantum embedding has occurred
# The output of this function will serve as the input for the RY gates in the layerwise quantum embedding ansatz
def param_shift():

    return theta_dash

In [7]:
qc = QuantumCircuit(13)
n_qubits = 12
n_layers = 3
theta = np.pi/4
for i in range(n_qubits):
    qc.h(i)
for l in range(0, n_layers):
    for j in range(n_qubits):
        qc.rx(theta, j)
        #qc.cx(j, j+1)
    for m in range(n_qubits-1):
        qc.cz(m, m+1)
    for k in range(n_qubits):
        qc.ry(theta, k)
print(qc)

      ┌───┐┌─────────┐   ┌─────────┐┌─────────┐                      »
 q_0: ┤ H ├┤ Rx(π/4) ├─■─┤ Ry(π/4) ├┤ Rx(π/4) ├────────────────■─────»
      ├───┤├─────────┤ │ └─────────┘├─────────┤┌─────────┐     │     »
 q_1: ┤ H ├┤ Rx(π/4) ├─■──────■─────┤ Ry(π/4) ├┤ Rx(π/4) ├─────■─────»
      ├───┤├─────────┤        │     └─────────┘├─────────┤┌─────────┐»
 q_2: ┤ H ├┤ Rx(π/4) ├────────■──────────■─────┤ Ry(π/4) ├┤ Rx(π/4) ├»
      ├───┤├─────────┤                   │     └─────────┘├─────────┤»
 q_3: ┤ H ├┤ Rx(π/4) ├───────────────────■──────────■─────┤ Ry(π/4) ├»
      ├───┤├─────────┤                              │     └─────────┘»
 q_4: ┤ H ├┤ Rx(π/4) ├──────────────────────────────■──────────■─────»
      ├───┤├─────────┤                                         │     »
 q_5: ┤ H ├┤ Rx(π/4) ├─────────────────────────────────────────■─────»
      ├───┤├─────────┤                                               »
 q_6: ┤ H ├┤ Rx(π/4) ├───────────────────────────────────────────────»
      